In [1]:
import json
from pathlib import Path
import shutil
import datetime as dt
from pprint import pprint
import pickle


import pandas as pd

from wattile.entry_point import init_logging
from wattile.data_reading import read_dataset_from_file
from wattile.buildings_processing import prep_for_rnn
from wattile.models import ModelFactory


PROJECT_DIRECTORY = Path().resolve().parents[1]


PROJECT_DIRECTORY = /Users/heslinge/Repos/Wattile/wattile


In [2]:
"""
This demo runs predictions on a pre-trained alpha model in ./tests/fixtures/alfa_exp_dir.
"""
exp_dir = PROJECT_DIRECTORY / "tests/fixtures/FTLB_FTLBCHWMeterCHWEnergyRate_r1"
with open(exp_dir / "configs.json", "r") as f:
    configs = json.load(f)

pprint(configs)

{'building': 'FTLB',
 'code_dir': '/projects/intelcamp/repos/WattChoice',
 'data_input': {'data_config': 'FTLB Config.json',
                'data_dir': '/projects/intelcamp/repos/WattChoice/../../data/STM '
                            'Campus',
                'end_time': '2023-04-01T00:00:00-07:00',
                'predictor_columns': [],
                'start_time': '2020-07-01T00:00:00-07:00',
                'target_var': 'FTLB CHW Meter CHW Energy Rate'},
 'data_output': {'exp_dir': '/projects/intelcamp/repos/WattChoice/results/FTLB/FTLB_FTLBCHWMeterCHWEnergyRate_r1',
                 'plot_comparison': True,
                 'plot_comparison_portion_end': 1.0,
                 'plot_comparison_portion_start': 0.0},
 'data_path': '../../data/STM Campus',
 'data_processing': {'data_split': '80:20:0',
                     'feat_stats': {'active': True, 'window_width': '15min'},
                     'feat_time': {'day_of_week': ['binary_reg',
                                      

In [3]:
exp_dir = PROJECT_DIRECTORY / "tests/fixtures/FTLB_FTLBCHWMeterCHWEnergyRate_r1"
with open(exp_dir / "predictors_target_config.json", "r") as f:
    boo = json.load(f)

print([x["column"] for x in boo["predictors"]])


['FTLB CHW Meter CHWST', 'FTLB HW Meter HWST', 'SRRL BMS Dew Point Temperature', 'SRRL BMS Diffuse Horizontal Irradiance', 'SRRL BMS Direct Normal Irradiance', 'SRRL BMS Dry Bulb Temperature', 'SRRL BMS Global Horizontal Irradiance', 'SRRL BMS Rainfall', 'SRRL BMS Relative Humidity', 'SRRL BMS Snow Depth', 'SRRL BMS Wet Bulb Temperature', "SRRL BMS Wind Direction at 19'", "SRRL BMS Wind Speed at 19'"]


In [4]:
"""
Init the model with it's configs. Be sure to switch it's use case to prediction.
"""

configs["data_output"]["exp_dir"] = exp_dir
configs["learning_algorithm"]["use_case"] = "prediction"
init_logging(local_results_dir=configs["data_output"]["exp_dir"])
model = ModelFactory.create_model(configs)

model

Logging to: /Users/heslinge/Repos/Wattile/tests/fixtures/FTLB_FTLBCHWMeterCHWEnergyRate_r1/output.out, PID: 67333


In [8]:
with open("test_predictor_data.pickle", "rb") as f:
    f.seek(0)
    data = pickle.load(f)

train_df, val_df = prep_for_rnn(configs, data)

val_df

,FTLB CHW Meter CHWST_min,FTLB HW Meter HWST_min,SRRL BMS Dew Point Temperature_min,SRRL BMS Diffuse Horizontal Irradiance_min,SRRL BMS Direct Normal Irradiance_min,SRRL BMS Dry Bulb Temperature_min,SRRL BMS Global Horizontal Irradiance_min,SRRL BMS Rainfall_min,SRRL BMS Relative Humidity_min,SRRL BMS Snow Depth_min,...,DOW_binary_fuzzy_0_mean,DOW_binary_fuzzy_1_mean,DOW_binary_fuzzy_2_mean,DOW_binary_fuzzy_3_mean,DOW_binary_fuzzy_4_mean,DOW_binary_fuzzy_5_mean,DOW_binary_fuzzy_6_mean,sin_MOY_mean,cos_MOY_mean,FTLB CHW Meter CHW Energy Rate
ts,,,,,,,,,,,,,,,,,,,,,
2022-12-31 18:00:00-07:00,47.208561,170.305649,28.071018,0.0,0.000000,34.309399,0.0,0.0,74.669998,7.066929,...,0.0,0.0,0.0,0.0,0.0,0.250000,0.750000,6.432491e-16,1.000000,-999
2022-12-31 18:15:00-07:00,47.098923,168.110992,27.793409,0.0,0.000000,34.557800,0.0,0.0,73.120003,7.070866,...,0.0,0.0,0.0,0.0,0.0,0.250000,0.750000,6.432491e-16,1.000000,-999
2022-12-31 18:30:00-07:00,47.250946,168.441498,27.956015,0.0,0.000000,35.058201,0.0,0.0,69.349998,7.047244,...,0.0,0.0,0.0,0.0,0.0,0.250000,0.750000,6.432491e-16,1.000000,-999
2022-12-31 18:45:00-07:00,47.266205,167.243805,28.037127,0.0,0.000000,36.514400,0.0,0.0,68.900002,7.047244,...,0.0,0.0,0.0,0.0,0.0,0.250000,0.750000,6.432491e-16,1.000000,-999
2022-12-31 19:00:00-07:00,47.228340,167.146454,28.098965,0.0,0.000000,35.952801,0.0,0.0,68.680000,6.996063,...,0.0,0.0,0.0,0.0,0.0,0.208333,0.791667,6.432491e-16,1.000000,-999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-01 22:45:00-07:00,47.231167,161.048874,27.295630,0.0,0.115112,27.437000,0.0,0.0,99.199997,6.897638,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.083333,1.721336e-02,0.999852,-999
2023-01-01 23:00:00-07:00,47.683289,166.150162,27.369343,0.0,0.308708,27.510799,0.0,0.0,99.300003,6.893701,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.041667,1.721336e-02,0.999852,-999
2023-01-01 23:15:00-07:00,47.425583,165.067795,27.286589,0.0,0.198830,27.471201,0.0,0.0,99.099998,6.893701,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.041667,1.721336e-02,0.999852,-999


In [9]:
pprint(configs["input_dim"])


237


In [10]:
"""
One line in, one prediction (with many quantiles) out.
"""
results = model.predict(val_df)

results

val_df:                            FTLB CHW Meter CHWST_min  FTLB HW Meter HWST_min  \
ts                                                                            
2022-12-31 18:00:00-07:00                 47.208561              170.305649   
2022-12-31 18:15:00-07:00                 47.098923              168.110992   
2022-12-31 18:30:00-07:00                 47.250946              168.441498   
2022-12-31 18:45:00-07:00                 47.266205              167.243805   
2022-12-31 19:00:00-07:00                 47.228340              167.146454   
...                                             ...                     ...   
2023-01-01 22:45:00-07:00                 47.231167              161.048874   
2023-01-01 23:00:00-07:00                 47.683289              166.150162   
2023-01-01 23:15:00-07:00                 47.425583              165.067795   
2023-01-01 23:30:00-07:00                 47.448753              166.308289   
2023-01-01 23:45:00-07:00                 47

<xarray.DataArray (timestamp: 120, quantile: 9, horizon: 1)>
array([[[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]],

       [[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]],

       [[nan],
        [nan],
        [nan],
        ...,
...
        ...,
        [nan],
        [nan],
        [nan]],

       [[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]],

       [[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]]], dtype=float32)
Coordinates:
  * timestamp  (timestamp) object 2022-12-31T18:00:00-07:00 ... 2023-01-01T23...
  * quantile   (quantile) float64 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9
  * horizon    (horizon) timedelta64[ns] 00:00:00